In [ ]:
#import of packages

import numpy as np
import math
import matplotlib.pyplot as plt

from scipy.linalg import sqrtm #matrix square root
from scipy.stats import gaussian_kde

print("ok")

In [2]:
def skripts_work():
  return "Monte carlo skript functions integrated."

In [ ]:
#elementary functions for monte carlo simulations


#fast monte carlo simulation with fixed QV

def fast_monte_carlo_simulation_with_fixed_QV(Sigma_true, Sigma_true_denoised, sampling_interval, drift, L_lambda=6,
                                              number_of_simulations=100, number_error_types=12, progress_number=50):
    """
    Simulate number_of_simulation times paths with Sigma_true, drift and sampling_interval. Always calculate rv an prv.
    Use the fast functions to do so. Then returns properties of the two estimators.

    Parameters
    ----------
    Sigma_true : numpy.ndarray
        True QV of the setting.
    Sigma_true_denoised : numpy.ndarray
        True QV of the setting without vanishing components in the SVD.
    sampling_interval : float
        Time diffference between observations on [o,1].
    drift : numpy.ndarray
        True drift of the setting.
    L_lambda : int
        The hyperparameter L_lambda defines the number of subsamples used in the choice of the tuning parameter lambda.
    number_of_simulations : int
        Number of simulated log-prices that are analysed with the estimators.
    number_error_types : int
        Number of different errors returned by this function.
    progress_number : int
        The integer decides after how many simulations the progress is printed.

    Returns
    -------
    eigenvalues_rv : numpy.ndarray
        Each row is an array of the eigenvalues of the rv for the corresponding simulation. They are sorted decreasingly.
    eigenvalues_prv : numpy.ndarray
        Each row is an array of the eigenvalues of the prv for the corresponding simulation. They are sorted decreasingly.


    lambdas_used : numpy.ndarray
        Array containing the used tuning parameters lambda of the prv estimators for each simulation.


    rank_rv : numpy.ndarray
        Array containing the ranks of the rv estimators for each simulation.
    rank_prv : numpy.ndarray
        Array containing the ranks of the prv estimators for each simulation.

    all_errors : numpy.ndarray
        A matrix with several columns, each row describes a simulation and each column an error.
        In this order, the different columns contain:

        -squared_2_norm_error_rv : The squared Frobenius norm errors of the rv estimators for each simulation.
        -squared_2_norm_error_prv : The squared Frobenius norm errors of the prv estimators for each simulation.
        -1_norm_error_rv : The nuclear norm errors of the rv estimators for each simulation.
        -1_norm_error_prv : The nuclear norm errors of the prv estimators for each simulation.
        -spectral_norm_error_rv : The spectral norm errors of the rv estimators for each simulation.
        -spectral_norm_error_prv : The spectral norm errors of the prv estimators for each simulation.

        -denoised_squared_2_norm_error_rv : The squared Frobenius norm errors of the rv estimators for each simulation
        with Sigma_true_denoised as inference target.
        -denoised_squared_2_norm_error_prv : The squared Frobenius norm errors of the prv estimators for each simulation
        with Sigma_true_denoised as inference target.
        -denoised_1_norm_error_rv : The nuclear norm errors of the rv estimators for each simulation
        with Sigma_true_denoised as inference target.
        -denoised_1_norm_error_prv : The nuclear norm errors of the prv estimators for each simulation
        with Sigma_true_denoised as inference target.
        -denoised_spectral_norm_error_rv : The spectral norm errors of the rv estimators for each simulation
        with Sigma_true_denoised as inference target.
        -denoised_spectral_norm_error_prv : The spectral norm errors of the prv estimators for each simulation
        with Sigma_true_denoised as inference target.
    """
    d=Sigma_true.shape[0]
    vola=sqrtm(Sigma_true)

    #arrays of returns
    eigenvalues_rv = np.zeros((number_of_simulations, d))
    eigenvalues_prv = np.zeros((number_of_simulations, d))
    lambdas_used = np.zeros(number_of_simulations)
    rank_rv = np.zeros(number_of_simulations)
    rank_prv = np.zeros(number_of_simulations)
    all_errors = np.zeros((number_of_simulations, number_error_types))


    for i in range(0,number_of_simulations):
        returns=fast_increments_generator(volatility=vola, drift=drift, sampling_interval=sampling_interval)

        rv=fast_realized_variance(returns)
        lambda_used=lambda_selection_via_subsampling_procedure(returns, L_lambda=L_lambda)
        prv, rank_of_prv=penalized_realized_variance_for_given_lambda_and_RV(rv, lambda_used)

        eigenvalues_rv[i,:] = np.sort(np.linalg.eigh(rv)[0])[::-1] #sort creates increasing, now decreasing
        eigenvalues_prv[i,:] = np.sort(np.linalg.eigh(prv)[0])[::-1] #sort creates increasing, now decreasing
        lambdas_used[i] = lambda_used
        rank_rv[i] = np.linalg.matrix_rank(rv)
        rank_prv[i] = rank_of_prv

        all_errors[i,0] = np.linalg.norm(rv-Sigma_true, 'fro')**2 #fro=Frobenius
        all_errors[i,1] = np.linalg.norm(prv-Sigma_true, 'fro')**2
        all_errors[i,2] = np.linalg.norm(rv-Sigma_true, 'nuc')# nuc=Nuclear
        all_errors[i,3] = np.linalg.norm(prv-Sigma_true, 'nuc')
        all_errors[i,4] = np.linalg.norm(rv-Sigma_true, 2)# 2=Spectral
        all_errors[i,5] = np.linalg.norm(prv-Sigma_true, 2)

        all_errors[i,6] = np.linalg.norm(rv-Sigma_true_denoised, 'fro')**2 #fro=Frobenius
        all_errors[i,7] = np.linalg.norm(prv-Sigma_true_denoised, 'fro')**2
        all_errors[i,8] = np.linalg.norm(rv-Sigma_true_denoised, 'nuc')# nuc=Nuclear
        all_errors[i,9] = np.linalg.norm(prv-Sigma_true_denoised, 'nuc')
        all_errors[i,10] = np.linalg.norm(rv-Sigma_true_denoised, 2)# 2=Spectral
        all_errors[i,11] = np.linalg.norm(prv-Sigma_true_denoised, 2)

        if i%progress_number==0:
            print(f'{i} simulations: finished.')

    return eigenvalues_rv , eigenvalues_prv , lambdas_used , rank_rv , rank_prv , all_errors


print("ok")


In [ ]:
#functions for sensitivity analysis of tuning parameters


#L_lambda

#function that finds all divisors of n

def divisors(n):
    """
    Returns all positive divisors of n.

    Parameters
    ----------
    n : int
        Integer for which divisors are calculated.

    Returns
    ----------
    divisors : numpy.ndarray
        Array containing all divisors.
    """
    small = []
    large = []

    for i in range(1, int(np.sqrt(n)) + 1):
        if n % i == 0:
            small.append(i)
            if i != n // i:    #n//i is division and than floored , != is not equal
                large.append(n // i)

    return np.array(small + large[::-1])


#fast sensitivity analysis of L_lambda

def fast_sensitivity_analysis_for_L_lambda_with_fixed_QV(Sigma_true, Sigma_true_denoised, sampling_interval, drift,
                                              number_of_simulations=100, number_error_types=2, progress_number=50):
    """
    Evaluate the influence of the hyperparameter L_lambda by using different values
    of it in the calculation of the PRV estimator.
    Then, return the final tuning parameters lambda and the rank and estimation errors of the PRV estimator
    in the squared Frobenius norm .

    Parameters
    ----------
    Sigma_true : numpy.ndarray
        True QV of the setting.
    Sigma_true_denoised : numpy.ndarray
        True QV of the setting without vanishing components in the SVD.
    sampling_interval : float
        Time diffference between observations on [o,1].
    drift : numpy.ndarray
        True drift of the setting.
    number_of_simulations : int
        Number of simulated log-prices that are analysed with the estimators.
    number_error_types : int
        Number of different errors returned by this function.
    progress_number : int
        The integer decides after how many simulations the progress is printed.

    Returns
    -------

    L_lambdas : numpy.ndarray
        Vector containing all analyzed L_lambdas.

    lambdas_used : numpy.ndarray
        Matrix containing the used tuning parameters lambda of the prv estimators.
        Each simulation is a row, each L_lambda choice corresponds to a column.

    rank_prv : numpy.ndarray
        Matrix containing the ranks of the prv estimators.
        Each simulation is a row, each L_lambda choice corresponds to a column.

    squared_2_norm_error_prv : numpy.ndarray
        Matrix containing the squared Frobenius norm errors of the prv estimators.
        Each simulation is a row, each L_lambda choice corresponds to a column.
    denoised_squared_2_norm_error_prv : numpy.ndarray
        Matrix containing the squared Frobenius norm errors of the prv estimators with Sigma_true_denoised as inference target.
        Each simulation is a row, each L_lambda choice corresponds to a column.

    """

    d=Sigma_true.shape[0]
    vola=sqrtm(Sigma_true)

    n=int(np.floor(1/sampling_interval))
    L_lambdas=divisors(n)

    #arrays of returns
    lambdas_used = np.zeros((number_of_simulations, len(L_lambdas)))
    rank_prv = np.zeros((number_of_simulations, len(L_lambdas)))
    squared_2_norm_error_prv = np.zeros((number_of_simulations, len(L_lambdas)))
    denoised_squared_2_norm_error_prv = np.zeros((number_of_simulations, len(L_lambdas)))


    for i in range(0,number_of_simulations):
        returns=fast_increments_generator(volatility=vola, drift=drift, sampling_interval=sampling_interval)

        rv=fast_realized_variance(returns)

        for z in range(len(L_lambdas)):
            lambda_used=lambda_selection_via_subsampling_procedure(returns, L_lambda=L_lambdas[z])
            prv, rank_of_prv=penalized_realized_variance_for_given_lambda_and_RV(rv, lambda_used)

            lambdas_used[i,z] = lambda_used
            rank_prv[i,z] = rank_of_prv

            squared_2_norm_error_prv[i,z] = np.linalg.norm(prv-Sigma_true, 'fro')**2
            denoised_squared_2_norm_error_prv[i,z] = np.linalg.norm(prv-Sigma_true_denoised, 'fro')**2


        if i%progress_number==0:
            print(f'{i} simulations: finished.')

    return L_lambdas , lambdas_used , rank_prv , squared_2_norm_error_prv , denoised_squared_2_norm_error_prv


#tuning parameter lambda

#fast sensitivity analysis of lambda

def fast_sensitivity_analysis_for_lambda_with_fixed_QV(Sigma_true, Sigma_true_denoised, sampling_interval, drift,
                                                       lambda_prefactors, L_lambda=6,
                                                       number_of_simulations=100, number_error_types=2, progress_number=50):
    """
    Evaluate the influence of thetuning parameter lambda by using different values
    of it in the calculation of the PRV estimator.
    Then, return the rank and the estimation errors of the PRV estimator.

    Parameters
    ----------
    Sigma_true : numpy.ndarray
        True QV of the setting.
    Sigma_true_denoised : numpy.ndarray
        True QV of the setting without vanishing components in the SVD.
    sampling_interval : float
        Time diffference between observations on [o,1].
    drift : numpy.ndarray
        True drift of the setting.
    lambda_prefactors : numpy.ndarray
        Vector containing all the prefactors that are multiplied with the actual lambda_subsam value to get a tuning parameter.
    L_lambda : int
        The hyperparameter L_lambda defines the number of subsamples used in the choice of the tuning parameter lambda.
    number_of_simulations : int
        Number of simulated log-prices that are analysed with the estimators.
    number_error_types : int
        Number of different errors returned by this function.
    progress_number : int
        The integer decides after how many simulations the progress is printed.

    Returns
    -------

    lambdas_used : numpy.ndarray
        Matrix containing the used tuning parameters lambda of the prv estimators.
        Each simulation is a row, each lambda prefactor corresponds to a column.

    rank_prv : numpy.ndarray
        Matrix containing the ranks of the prv estimators.
        Each simulation is a row, each lambda prefactor corresponds to a column.

    squared_2_norm_error_prv : numpy.ndarray
        Matrix containing the squared Frobenius norm errors of the prv estimators.
        Each simulation is a row, each lambda prefactor corresponds to a column.
    denoised_squared_2_norm_error_prv : numpy.ndarray
        Matrix containing the squared Frobenius norm errors of the prv estimators with Sigma_true_denoised as inference target.
        Each simulation is a row, each lambda prefactor corresponds to a column.

    """

    d=Sigma_true.shape[0]
    vola=sqrtm(Sigma_true)

    #arrays of returns
    lambdas_used = np.zeros((number_of_simulations, len(lambda_prefactors)))
    rank_prv = np.zeros((number_of_simulations, len(lambda_prefactors)))
    squared_2_norm_error_prv = np.zeros((number_of_simulations, len(lambda_prefactors)))
    denoised_squared_2_norm_error_prv = np.zeros((number_of_simulations, len(lambda_prefactors)))


    for i in range(0,number_of_simulations):
        returns=fast_increments_generator(volatility=vola, drift=drift, sampling_interval=sampling_interval)

        rv=fast_realized_variance(returns)
        lambda_used=lambda_selection_via_subsampling_procedure(returns, L_lambda=L_lambda)

        for z in range(len(lambda_prefactors)):
            prv, rank_of_prv=penalized_realized_variance_for_given_lambda_and_RV(rv, lambda_prefactors[z]*lambda_used)

            lambdas_used[i,z] = lambda_prefactors[z]*lambda_used
            rank_prv[i,z] = rank_of_prv

            squared_2_norm_error_prv[i,z] = np.linalg.norm(prv-Sigma_true, 'fro')**2
            denoised_squared_2_norm_error_prv[i,z] = np.linalg.norm(prv-Sigma_true_denoised, 'fro')**2


        if i%progress_number==0:
            print(f'{i} simulations: finished.')

    return lambdas_used , rank_prv , squared_2_norm_error_prv , denoised_squared_2_norm_error_prv


print("ok")

In [ ]:
#functions for comparison of theoretical findings with simulations

#formulas for the tuning parameters

def lambda_esterr_calculation(Sigma, drift, sampling_interval, tau, gamma= 82*10**(6)):

    """
    Calculates the positive tuning parameter lambda_esterr from formula (7.9).

    Parameters
    ----------
    Sigma : numpy.ndarray
        True QV of the setting.
    drift : numpy.ndarray
        True drift of the setting.
    sampling_interval : float
        Time diffference between observations on [o,1].
    tau : float
        Positive argument of lambda_esterr in (7.9).
    gamma : float
        Absolute constant gamma from Theorem 4.7.

    Returns
    -------
    lambda_esterr : float
        Positive tuning parameter lambda_esterr from formula (7.9).
    index_of_maximal_value : int
        Position of the argument in the maximum of lambda_esterr in (7.9) that attains the maximum (0,1,2 or 3).

    """
    v_mu=np.linalg.norm(drift)**2
    v_c_2=np.trace(Sigma)
    #v_c_2=np.linalg.norm(sqrtm(Sigma), 'fro')**2   # 'fro' is Frobenius norm
    v_c_inf=np.linalg.norm(Sigma, 2)   # 2 is spectral norm
    d=Sigma.shape[0]

    lambda_esterr=max(20*(v_c_2*v_mu*sampling_interval)/v_c_inf, 20*(v_c_2*v_mu*sampling_interval)**0.5,
                     ((4*gamma*(tau+np.log(6*d)))*(v_c_2*v_c_inf*sampling_interval))**0.5,
                     (2*gamma/2255)*(tau+np.log(6*d))*v_c_2*sampling_interval)

    index_of_maximal_value=np.argmax(np.array([220*(v_c_2*v_mu*sampling_interval)/v_c_inf, 20*(v_c_2*v_mu*sampling_interval)**0.5,
                     ((4*gamma*(tau+np.log(6*d)))*(v_c_2*v_c_inf*sampling_interval))**0.5,
                     (2*gamma/2255)*(tau+np.log(6*d))*v_c_2*sampling_interval]))

    return lambda_esterr , index_of_maximal_value

def lambda_conint_calculation(Sigma, drift, sampling_interval, alpha=0.05, gamma= 82*10**(6)):

    """
    Calculates the positive tuning parameter lambda_conint from formula (7.11).

    Parameters
    ----------
    Sigma : numpy.ndarray
        True QV of the setting.
    drift : numpy.ndarray
        True drift of the setting.
    sampling_interval : float
        Time diffference between observations on [o,1].
    alpha : float
        Positive argument alpha in (0,1) of lambda_conint in (7.11). It is 1 minus the confidence level.
    gamma : float
        Absolute constant gamma from Theorem 4.7.

    Returns
    -------
    lambda_esterr : float
        Positive tuning parameter lambda_conint from formula (7.11).
    index_of_maximal_value : int
        Position of the argument in the maximum of lambda_conint in (7.11) that attains the maximum (0,1,2 or 3).

    """
    v_mu=np.linalg.norm(drift)**2
    v_c_2=np.trace(Sigma)
    #v_c_2=np.linalg.norm(sqrtm(Sigma), 'fro')**2   # 'fro' is Frobenius norm
    v_c_inf=np.linalg.norm(Sigma, 2)   # 2 is spectral norm
    d=Sigma.shape[0]

    lambda_conint=max(20*(v_c_2*v_mu*sampling_interval)**0.5,
                     ((4*gamma*np.log(6*d/alpha))*(v_c_2*v_c_inf*sampling_interval))**0.5,
                      20*(v_c_2*v_mu*sampling_interval)/v_c_inf,
                     (2*gamma/2255)*np.log(6*d/alpha)*v_c_2*sampling_interval)

    index_of_maximal_value=np.argmax(np.array([20*(v_c_2*v_mu*sampling_interval)**0.5,
                     ((4*gamma*np.log(6*d/alpha))*(v_c_2*v_c_inf*sampling_interval))**0.5,
                      20*(v_c_2*v_mu*sampling_interval)/v_c_inf,
                     (2*gamma/2255)*np.log(6*d/alpha)*v_c_2*sampling_interval]))

    return lambda_conint , index_of_maximal_value



#fast monte carlo simulation for the comparison with theory

def monte_carlo_simulation_for_theory(Sigma_true, Sigma_true_denoised, sampling_interval, drift, tau, L_lambda=6,
                                      alpha=0.05, gamma= 82*10**(6),
                                      number_of_simulations=100, number_error_types=12, progress_number=50):
    """
    Simulate number_of_simulation times paths with Sigma_true, drift and sampling_interval.
    Then, determine for each path lambda-subsam. Always calculate the prv for lambda_subsam, lambda_esterr and lambda_conint.
    Then returns properties of the 3 estimators (3 different prv estimators, one per choice of lambda).

    Parameters
    ----------
    Sigma_true : numpy.ndarray
        True QV of the setting.
    Sigma_true_denoised : numpy.ndarray
        True QV of the setting without vanishing components in the SVD.
    sampling_interval : float
        Time diffference between observations on [o,1].
    drift : numpy.ndarray
        True drift of the setting.
    tau : float
        Positive argument of lambda_esterr in (7.9).
    L_lambda : int
        The hyperparameter L_lambda defines the number of subsamples used in the choice of the tuning parameter lambda.
    alpha : float
        Positive argument alpha in (0,1) of lambda_conint in (7.11). It is 1 minus the confidence level.
    gamma : float
        Absolute constant gamma from Theorem 4.7.
    number_of_simulations : int
        Number of simulated log-prices that are analysed with the estimators.
    number_error_types : int
        Number of different errors returned by this function.
    progress_number : int
        The integer decides after how many simulations the progress is printed.

    Returns
    -------

    lambdas_subsam_used : numpy.ndarray
        Array containing the used tuning parameters lambda_subsam of the prv estimators for each path.
    lambda_esterr : float
        Tuning parameter lambda_esterr of the prv estimators used in every path.
    index_of_maximal_value_esterr : int
        Position of the argument in the maximum of lambda_esterr in (7.9) that attains the maximum (0,1,2 or 3).
    lambda_conint : float
        Tuning parameter lambda_conint of the prv estimators used in every path.
    index_of_maximal_value_conint : int
        Position of the argument in the maximum of lambda_conint in (7.11) that attains the maximum (0,1,2 or 3).

    eigenvalues_rv : numpy.ndarray
        Each row is an array of the eigenvalues of the rv for the corresponding simulation. They are sorted decreasingly.
    eigenvalues_prv_subsam : numpy.ndarray
        Each row is an array of the eigenvalues of the prv for the corresponding simulation. They are sorted decreasingly.
    eigenvalues_prv_esterr : numpy.ndarray
        Each row is an array of the eigenvalues of the prv for the corresponding simulation. They are sorted decreasingly.
    eigenvalues_prv_conint : numpy.ndarray
        Each row is an array of the eigenvalues of the prv for the corresponding simulation. They are sorted decreasingly.


    rank_rv : numpy.ndarray
        Array containing the ranks of the rv estimators for each simulation.
    rank_prv : numpy.ndarray
        Matrix containing the ranks of the prv estimators for each simulation.
        Each column belongs to a special choice of the tuning parameter: lambda_subsam, lambda_esterr and lambda_conint.

    all_errors : numpy.ndarray
        A matrix with several columns, each row describes a simulation and each column an error.
        In this order, the different columns contain:

        -squared_2_norm_error_rv : The squared Frobenius norm errors of the rv estimators for each simulation.
        -squared_2_norm_error_prv_subsam : The squared Frobenius norm errors of the prv estimators for each simulation.
        -squared_2_norm_error_prv_esterr : The squared Frobenius norm errors of the prv estimators for each simulation.
        -squared_2_norm_error_prv_conint : The squared Frobenius norm errors of the prv estimators for each simulation.

        -1_norm_error_rv : The nuclear norm errors of the rv estimators for each simulation.
        -1_norm_error_prv_subsam : The nuclear norm errors of the prv estimators for each simulation.
        -1_norm_error_prv_esterr : The nuclear norm errors of the prv estimators for each simulation.
        -1_norm_error_prv_conint : The nuclear norm errors of the prv estimators for each simulation.

        -spectral_norm_error_rv : The spectral norm errors of the rv estimators for each simulation.
        -spectral_norm_error_prv_subsam : The spectral norm errors of the prv estimators for each simulation.
        -spectral_norm_error_prv_esterr : The spectral norm errors of the prv estimators for each simulation.
        -spectral_norm_error_prv_conint : The spectral norm errors of the prv estimators for each simulation.

    """
    d=Sigma_true.shape[0]
    vola=sqrtm(Sigma_true)

    lambda_esterr, index_of_maximal_value_esterr=lambda_esterr_calculation(Sigma_true, drift, sampling_interval, tau, gamma)
    lambda_conint, index_of_maximal_value_conint=lambda_conint_calculation(Sigma_true, drift, sampling_interval, alpha, gamma)

    #arrays of returns
    lambdas_subsam_used = np.zeros(number_of_simulations)

    eigenvalues_rv = np.zeros((number_of_simulations, d))
    eigenvalues_prv_subsam = np.zeros((number_of_simulations, d))
    eigenvalues_prv_esterr = np.zeros((number_of_simulations, d))
    eigenvalues_prv_conint = np.zeros((number_of_simulations, d))

    rank_rv = np.zeros(number_of_simulations)
    rank_prv = np.zeros((number_of_simulations, 3))
    all_errors = np.zeros((number_of_simulations, number_error_types))


    for i in range(0,number_of_simulations):
        returns=fast_increments_generator(volatility=vola, drift=drift, sampling_interval=sampling_interval)

        rv=fast_realized_variance(returns)
        lambda_subsam_used=lambda_selection_via_subsampling_procedure(returns, L_lambda=L_lambda)
        prv_subsam, rank_of_prv_subsam=penalized_realized_variance_for_given_lambda_and_RV(rv, lambda_subsam_used)

        prv_esterr, rank_of_prv_esterr=penalized_realized_variance_for_given_lambda_and_RV(rv, lambda_esterr)
        prv_conint, rank_of_prv_conint=penalized_realized_variance_for_given_lambda_and_RV(rv, lambda_conint)

        eigenvalues_rv[i,:] = np.sort(np.linalg.eigh(rv)[0])[::-1] #sort creates increasing, now decreasing
        eigenvalues_prv_subsam[i,:] = np.sort(np.linalg.eigh(prv_subsam)[0])[::-1] #sort creates increasing, now decreasing
        eigenvalues_prv_esterr[i,:] = np.sort(np.linalg.eigh(prv_esterr)[0])[::-1] #sort creates increasing, now decreasing
        eigenvalues_prv_conint[i,:] = np.sort(np.linalg.eigh(prv_conint)[0])[::-1] #sort creates increasing, now decreasing

        lambdas_subsam_used[i] = lambda_subsam_used

        rank_rv[i] = np.linalg.matrix_rank(rv)
        rank_prv[i,0] = rank_of_prv_subsam
        rank_prv[i,1] = rank_of_prv_esterr
        rank_prv[i,2] = rank_of_prv_conint

        all_errors[i,0] = np.linalg.norm(rv-Sigma_true, 'fro')**2 #fro=Frobenius
        all_errors[i,1] = np.linalg.norm(prv_subsam-Sigma_true, 'fro')**2
        all_errors[i,2] = np.linalg.norm(prv_esterr-Sigma_true, 'fro')**2
        all_errors[i,3] = np.linalg.norm(prv_conint-Sigma_true, 'fro')**2

        all_errors[i,4] = np.linalg.norm(rv-Sigma_true, 'nuc')# nuc=Nuclear
        all_errors[i,5] = np.linalg.norm(prv_subsam-Sigma_true, 'nuc')
        all_errors[i,6] = np.linalg.norm(prv_esterr-Sigma_true, 'nuc')
        all_errors[i,7] = np.linalg.norm(prv_conint-Sigma_true, 'nuc')

        all_errors[i,8] = np.linalg.norm(rv-Sigma_true, 2)# 2=Spectral
        all_errors[i,9] = np.linalg.norm(prv_subsam-Sigma_true, 2)
        all_errors[i,10] = np.linalg.norm(prv_esterr-Sigma_true, 2)
        all_errors[i,11] = np.linalg.norm(prv_conint-Sigma_true, 2)


        if i%progress_number==0:
            print(f'{i} simulations: finished.')

    return (lambdas_subsam_used, lambda_esterr, index_of_maximal_value_esterr, lambda_conint, index_of_maximal_value_conint,
            eigenvalues_rv , eigenvalues_prv_subsam , eigenvalues_prv_esterr , eigenvalues_prv_conint ,
            rank_rv , rank_prv , all_errors)


print("ok")